### Модель [RealESRGAN](https://github.com/xinntao/Real-ESRGAN)

#### Установка зависимостей

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
from tqdm import tqdm
import os
from google.colab import files
from posix import O_SYNC
import shutil
import cv2

In [ ]:
def cut_frames_from_videos(videos_list: list[str], target_dir: str) -> None:
  '''
  Функция для кадрирования видео

  Input:
  videos_list - лист с путем до видео
  target_dir - директория сохранения изображаний


  '''
    for vid in tqdm(videos_list):
        cap = cv2.VideoCapture(vid)
        cap_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if cap_len == 0:
            break
        i = 0
        while True:
            success, frame = cap.read()
            if i % 1 != 0:
                i += 1
                continue
            if not success:
                break
            vid_name = vid.split('/')[-1].split('_')[0]
            save_path = os.path.join(target_dir, f'{vid_name}_{i}.jpg')
            cv2.imwrite(save_path, frame)
            i += 1

In [ ]:
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

In [ ]:
TARGET_DIR = '/content/Real-ESRGAN/upload'
if os.path.exists(TARGET_DIR):
  shutil.rmtree(TARGET_DIR)
  os.mkdir(TARGET_DIR)
else:
  os.mkdir(TARGET_DIR)
cut_frames_from_videos(['/content/0_144 (1).mp4'], TARGET_DIR)

100%|██████████| 1/1 [00:04<00:00,  4.61s/it]


In [ ]:
len(os.listdir(path=TARGET_DIR))

3600

### Inference

In [ ]:
!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 3.5 --face_enhance

In [ ]:
import cv2
import os

image_folder = '/content/Real-ESRGAN/results'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

### Запуск видео

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/Real-ESRGAN/video_1.avi"

# Compressed video path
compressed_path = "/content/Real-ESRGAN/video.avi"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)